In [1]:
import sys
sys.path.append('../')
from settings import *

import pandas as pd
import numpy as np

data_path_net = '../data/processed/MolecularNetwork/'
data_path_ms = '../data/processed/MS2MassBank/'

In [ ]:
import csv
import numpy as np
from matchms import Spectrum
spectrums_features=[]
with open('../data/raw/sample/DDA_Pos_EF.csv', 'r', encoding='utf-8-sig') as file:
    reader = csv.reader(file, delimiter=';')
    for row in reader:
        if row[0] == 'id' or not row[0]:
            continue
        mz_intensities = row[1].split(' ')
        mz_values, intensity_values = zip(*[mz_intensity.split(':') for mz_intensity in mz_intensities])
        spectrum = Spectrum(mz=np.array(list(map(float, mz_values))),
                            intensities=np.array(list(map(float, intensity_values))),
                            metadata={'id': row[0],
                                      'num_peaks': len(mz_values)})
        spectrums_features.append(spectrum)

In [ ]:
import matchms.filtering as ms_filters
from matchms import Spectrum

def peak_processing(spectrum):
    spectrum = ms_filters.default_filters(spectrum)
    spectrum = ms_filters.normalize_intensities(spectrum)
    spectrum = ms_filters.select_by_intensity(spectrum, intensity_from=0.05)
    return spectrum

def peak_transformation(spectrum):
    spectrum = Spectrum(mz=spectrum.mz, intensities=np.sqrt(spectrum.intensities) , metadata=spectrum.metadata)
    return spectrum

spectrums_features = [peak_processing(s) for s in spectrums_features]
spectrums_features = [peak_transformation(s) for s in spectrums_features]
spectrums_features=[s for s in spectrums_features if len(s.peaks)>=3]

In [ ]:
from matchms.exporting import save_as_msp
save_as_msp(spectrums_features, data_path_ms + MSrawdata + '_sample_features.msp')